# Dask tutorial

In [2]:
# PyArrow is a dependency of Dask
# !sudo apt-get update
# !sudo apt-get install python3-dask
# !pip3 install pyarrow

import dask
import dask.dataframe as dd

# When dask uses pandas 2.0.0+ it casts object columns to string automatically ([("i", 0.48)] -> '[("i", 0.48)]')
_ = dask.config.set({"dataframe.convert-string": False})

Dask Dataframes are just collections of Pandas-like Dataframes.

In [3]:
dataset_path = "smadex-challenge-predict-the-revenue/train/train"
filters = [("datetime", ">=", "2025-10-01-00-00"), ("datetime", "<", "2025-10-13-00-00")]

ddf = dd.read_parquet(
    dataset_path,
    filters = filters
)

We can operate with Dask Dataframes with the same API as the Pandas DataFrames...

In [13]:
ddf.head()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,None,NaN,NaN,NaN,None,None,None,None,0a7fbf18-5041-42af-bd0a-0cb6586b8598,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[6506b7e0a24666debd08f74266800f2eb154df5a, 150...",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[2b472e3dc96f1847490d7411b25e12ed417b9714, 3ba...",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[1031535cf2a1315422fd05d321349bcd3c3ffc04, 478...",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00


...but we cannot actually use them like this. Note that getting the header of the dataframe had a lot of computational cost because we had to load the data into memory.

In [14]:
ddf

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,registration,retention_d1_to_d7,retention_d3_to_d7,retention_d7_to_d14,retention_d1,retention_d3,retentiond7,advertiser_bundle,advertiser_category,advertiser_subcategory,advertiser_bottom_taxonomy_level,carrier,country,region,dev_make,dev_model,dev_os,dev_osv,hour,release_date,release_msrp,weekday,avg_act_days,avg_daily_sessions,avg_days_ins,avg_duration,bcat,bcat_bottom_taxonomy,bundles_cat,bundles_cat_bottom_taxonomy,bundles_ins,city_hist,country_hist,cpm,cpm_pct_rk,ctr,ctr_pct_rk,dev_language_hist,dev_osv_hist,first_request_ts,first_request_ts_bundle,first_request_ts_category_bottom_taxonomy,hour_ratio,iap_revenue_usd_bundle,iap_revenue_usd_category,iap_revenue_usd_category_bottom_taxonomy,last_buy,last_buy_ts_bundle,last_buy_ts_category,last_ins,last_install_ts_bundle,last_install_ts_category,advertiser_actions_action_count,advertiser_actions_action_last_timestamp,user_actions_bundles_action_count,user_actions_bundles_action_last_timestamp,last_advertiser_action,new_bundles,num_buys_bundle,num_buys_category,num_buys_category_bottom_taxonomy,region_hist,rev_by_adv,rwd_prank,user_bundles,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
npartitions=144,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,int32,int32,int32,int32,int64,int64,int64,float64,float64,float64,int32,int32,int32,int32,int32,int32,int32,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,int32,float64,object,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,object,object,object,object,object,object,int64,object,object,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,int32,float64,object,object,object,object,object,category[known]
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


We need to manually get hold of the partitions individually. We can do this easily by converting the DataFrame into a list of Dask Delayed objects.

In [7]:
parts = [part for part in ddf.to_delayed()]

Then load each part separately. Just loading one partition is not _too_ slow.

In [21]:
part_0 = parts[0].compute()
part_0.head()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,0,1,1,1,1,1,1,2.147718,2.147718,2.147718,...,"[88981729bd5c1e5aea9ada4bce00a2531e9e98f7, 25c...",0.019802,6.0,0.913366,None,None,None,None,819ecc0e-1a97-43ed-83f6-b9ede4f7fc48,2025-10-01-00-00
1,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,None,NaN,NaN,NaN,None,None,None,None,0a7fbf18-5041-42af-bd0a-0cb6586b8598,2025-10-01-00-00
2,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[6506b7e0a24666debd08f74266800f2eb154df5a, 150...",0.399021,6.0,0.999388,None,None,None,None,fc1a2689-b136-4ffa-b23b-9d8215bd720f,2025-10-01-00-00
3,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[2b472e3dc96f1847490d7411b25e12ed417b9714, 3ba...",0.121547,6.0,1.000000,None,None,None,None,0340fcc6-50bd-42ab-b9f4-4c1184b640cb,2025-10-01-00-00
4,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,...,"[1031535cf2a1315422fd05d321349bcd3c3ffc04, 478...",0.293285,6.0,0.160243,None,None,None,None,219d253f-bef4-4039-84b2-ed55f009cc43,2025-10-01-00-00


The loaded partition is a Pandas DataFrame.

In [20]:
type(part_0)

pandas.core.frame.DataFrame

In [19]:
part_0.describe()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,release_msrp,weekday,avg_act_days,avg_days_ins,first_request_ts,last_buy,last_ins,weekend_ratio,weeks_since_first_seen,wifi_ratio
count,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,121887.000000,...,109905.000000,121887.0,61107.000000,7183.000000,5.390700e+04,2.374000e+03,2.052200e+04,63305.000000,67489.000000,63304.000000
mean,0.031217,0.042301,0.044615,0.046601,0.095096,0.124853,0.174161,8.001834,8.308581,9.071367,...,564.108494,3.0,3.998519,5.228739,1.758773e+09,1.758279e+09,1.758424e+09,0.329158,4.877195,0.641166
std,0.173906,0.201277,0.206458,0.210783,0.996749,1.397024,2.388029,2048.960358,2049.186514,2050.423635,...,514.951762,0.0,2.115821,5.584633,3.757641e+05,7.122233e+05,7.406386e+05,0.270035,1.947677,0.398610
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,30.000000,3.0,1.000000,0.000000,1.758067e+09,1.756600e+09,1.756771e+09,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,180.000000,3.0,2.000000,1.000000,1.758499e+09,1.757735e+09,1.757857e+09,0.129870,4.000000,0.221200
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,299.000000,3.0,4.000000,3.333333,1.758499e+09,1.758478e+09,1.758611e+09,0.279810,6.000000,0.842471
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,899.000000,3.0,6.000000,7.000000,1.759190e+09,1.758888e+09,1.759091e+09,0.458333,6.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,136.000000,150.000000,368.000000,696622.140000,696622.140000,696622.140000,...,5160.000000,3.0,7.000000,28.000000,1.759190e+09,1.759104e+09,1.759277e+09,1.000000,6.000000,1.000000


In [24]:
part_0.isna()

,buyer_d1,buyer_d7,buyer_d14,buyer_d28,buy_d7,buy_d14,buy_d28,iap_revenue_d7,iap_revenue_d14,iap_revenue_d28,...,user_bundles_l28d,weekend_ratio,weeks_since_first_seen,wifi_ratio,whale_users_bundle_num_buys_prank,whale_users_bundle_revenue_prank,whale_users_bundle_total_num_buys,whale_users_bundle_total_revenue,row_id,datetime
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121882,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
121883,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False
121884,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,True,True,True,True,False,False
121885,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,True,True,False,False


## Using Dask with PyTorch

The main objective is to be able to use PyTorch's DataLoaders with our Dask DataFrame. This is difficult because PyTorch does not accept Dask DataFrames.

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils as utils

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

The _state-of-the-art_ solution (ChatGPT's suggestion) is to create a subclass of the DataLoader.

In [26]:
class DaskIterableDataset(utils.data.IterableDataset):
    """A PyTorch IterableDataset fitted to accept Dask DataFrames as input."""
    
    def __init__ (self, ddf: dd.DataFrame):
        self.ddf = ddf

    def __iter__ (self):
        for part in self.ddf.to_delayed():
            pdf = part.compute()
            for _, row in pdf.iterrows():
                yield row
            del pdf   # optional but safe

Then iterate over it. This takes a LOT of time, so we 

In [ ]:
dataset = DaskIterableDataset(ddf)

N = 1e5
for i, row in enumerate(dataset):
    if i % N == 0:
        print(row.values)

[0 1 1 1 1 1 1 2.147718 2.147718 2.147718 nan 1.0 1.0 1.0 1.0 0.0 1.0
 '26fc75598f3a2e63782b4166883034072c49b4aa' 'game' 'skill game' None None
 'us' None 'generic' 'android 16.0' 'android' '16.0.0' '00' '2025_july'
 nan 3 2.0 None 17.0 None list([('Game/Card', 241)])
 list([('Solitaire', 9)]) list([('Game/Card', 2)])
 list([('Solitaire', 1)])
 array(['d54f414516df4ba311d4b2a67b2d895175ea285b',
        '0de540d0711175dd3a4ffa7e7f8450abc7870328'], dtype=object)
 list([('US-CA-Modesto', 372), ('US-PA-Bovard', 2), ('US-CA-Tracy', 103), ('US-CA-Fresno', 54), ('US-PA-Youngstown', 3)])
 list([('US', 534)])
 list([('r', 324.06158940143393), ('i', 268.11353594014605)])
 list([('i', 0.8640278470866332), ('r', 0.8215736422446216)]) None None
 list([('en', 534)]) list([('15.0.0', 534)]) 1759190400.0
 list([('d026805c95a7fa27fcae2a72fd5d6d092b7653bf', 1758499200), ('43ea9ab81b256dc4c5ed3786e645bad87c3b6e93', 1759190400), ('047665c6af07408267630aecb9534676e55d4178', 1759104000), ('25cdfd75fffab2e19

[0 0 0 0 0 0 0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0
 'ff9bdd5dc427114eef1c6372526548e01a5ef45d' 'finance' 'finance'
 'investing & financial management (general)' None 'ng' None 'apple'
 'iphone 14 pro' 'ios' '17.5.1' '01' '2022_september' 1499.0 3 2.5
 list([('a5a357330e68f6583e87163b9aa063205391fa12', 1)]) nan
 list([('a5a357330e68f6583e87163b9aa063205391fa12', 2651)])
 list([('Games/Games/Adventure', 6), ('Games/Games/Casual', 143)])
 list([('Idle RPG', 6), ('Artillery Shooter', 143)])
 list([('Games/Games/Casual', 1), ('Games/Games/Adventure', 1)])
 list([('Artillery Shooter', 1), ('Idle RPG', 1)]) None None
 list([('HK', 149)])
 list([('b', 46.09741826923077), ('r', 14.619424116056797), ('i', 26.46763445660913)])
 list([('r', 0.314370777665527), ('i', 0.6973684210526315), ('b', 0.7944078947368421)])
 None None list([('zh', 149)]) list([('15.8.4', 149)]) 1758412800.0
 list([('5c0b36b58157c8185946e329ea180a435f235b4c', 1758412800), ('a5a357330e68f6583e87163b9aa063205391fa12', 1758

[0 0 0 0 0 0 0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 '3817dd799bec28bc68dcba7c6e6690a737af20cc' None None None
 'cloudflare warp' 'bt' 'thimphu district' 'oneplus' 'cph2487' 'android'
 '15.0.0' '06' '2023_february' 488.0 3 1.5 None nan None
 list([('Application/Tools', 20)]) list([('Cloud Storage', 20)])
 list([('Application/Tools', 1)]) list([('Cloud Storage', 1)]) None
 list([('BT-15-Thimphu', 17)]) list([('BT', 20)]) None None None None
 list([('en', 20)]) list([('15.0.0', 20)]) 1758153600.0
 list([('572f4ea557dbb90dad815064a616d7d7f76a939c', 1758153600)])
 list([('Cloud Storage', 1758153600)])
 list([(20, 0.05), (21, 0.3), (9, 0.3), (22, 0.25), (23, 0.1)]) None None
 None nan None None nan None None None None
 list([('32233e3b05ed51c8ee5deec8728fe79aa3686756', [('session', 3)])])
 list([('32233e3b05ed51c8ee5deec8728fe79aa3686756', [('session', 1757839212)])])
 None array(['572f4ea557dbb90dad815064a616d7d7f76a939c'], dtype=object)
 None None None list([('BT-34', 3), ('BT-15', 17)

[0 0 0 0 0 0 0 0.0 0.0 0.0 0.0 1.0 1.0 1.0 1.0 1.0 1.0
 '7eaeef21e215654ea75a5ef78692d1f8ffd0ca54' 'sport betting' 'unknown' None
 None 'ru' None 'samsung' 'sm-a045f' 'android' '14.0.0' '10'
 '2022_october' 227.0 3 nan None nan None None None None None None None
 None None None None None None None nan None None None None None None nan
 None None nan None None None None None None None None None None None None
 None None None None nan nan nan None None None None
 'cc81b633-5f8a-48b4-b1a3-f67bc0bafc55' '2025-10-01-10-00']
[0 0 0 0 0 0 0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 'db42749ef73fd488a95e37718f033eab132da07c' 'application' 'communication'
 'social messaging' 'windstream communications' 'us' 'new york' 'google'
 'pixel 8a' 'android' '15.0.0' '11' '2024_may' 559.0 3 nan None nan None
 None None None None None None None None None None None None None nan None
 None None None None None nan None None nan None None None None None None
 None None None None None None None None None None 

[0 0 0 0 0 0 0 0.0 0.0 0.0 nan 0.0 0.0 1.0 0.0 0.0 0.0
 'd5cdf45466c8edead05b468320028a02319c80cb' None None None 'home-connect'
 'za' 'gauteng' 'samsung' 'sm-a055f' 'android' '15.0.0' '13'
 '2023_october' nan 3 3.5 None nan None
 list([('Application/Communication', 8), ('Application/Video Players & Editors', 2), ('Application/Music & Audio', 14), ('Game/Trivia', 82)])
 list([('Browsers', 8), ('Video Editing', 2), ('Media Players', 14), ('Trivia', 82)])
 list([('Application/Music & Audio', 1), ('Application/Communication', 1), ('Game/Trivia', 1)])
 list([('Trivia', 1), ('Browsers', 1), ('Media Players', 1)])
 array(['cd9b2558bcf52078c64242e751017419651181d9'], dtype=object)
 list([('ZA-GP-Johannesburg', 213), ('ZA-FS-Bloemfontein', 4)])
 list([('ZA', 222)]) None None None None list([('en', 222)])
 list([('15.0.0', 222)]) 1758499200.0
 list([('f23c7f08f481fd287e18f33f2724ec7ebaea3b15', 1758499200)])
 list([('Trivia', 1758499200)])
 list([(5, 0.08290155440414508), (14, 0.0621761658031088

...

[1 1 1 1 1 1 1 0.5432643556200979 0.5432643556200979 0.5432643556200979
 nan 1.0 0.0 0.0 1.0 0.0 0.0 '1c88d1e1f99e2e2c35b97272ecf52c9a9e0a905c'
 'games' 'games/racing' 'action sandbox' 'novanet sp telecom' 'br'
 'são paulo' 'motorola' 'one fusion' 'android' '11.0.0' '23'
 '2019_september' 250.0 4 6.0
 list([('4a362b7368057cb3aeee27b06cc9621b2fe4176e', 1)]) 11.0
 list([('4a362b7368057cb3aeee27b06cc9621b2fe4176e', 36)])
 list([('Application/Social', 371), ('Application/Maps & Navigation', 2)])
 list([('Dating', 371), ('Bus & Rail Services', 2)])
 list([('Application/Social', 1)]) list([('Dating', 1)])
 array(['9a76cf31c46a6ae933a1ff369d2ed1fc36fe0a3b',
        '3a733cb011b8a90097c38782248e12b0a52838f8',
        'cd9b2558bcf52078c64242e751017419651181d9',
        '3817dd799bec28bc68dcba7c6e6690a737af20cc'], dtype=object)
 list([('BR-SP-Sao Paulo', 699)]) list([('BR', 699)]) None None None None
 list([('pt', 699)]) list([('11.0.0', 699)]) 1759104000.0
 list([('d2b686b27c116e21fc71f217d9ecf

[1 1 1 1 1 1 1 0.49 0.49 0.49 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 '3b565d52f6463cc295dc9c1f82f9c2eb3aed0a38' 'application' 'dating' None
 None 'mx' None 'xiaomi' '23129ra5fl' 'android' '14.0.0' '00'
 '2023_september' nan 5 7.0 None nan None
 list([('Application/Tools', 12), ('Application/Entertainment', 1), ('Application/Video Players & Editors', 64)])
 list([('Media Players', 65), ('Customization & Ringtones', 9), ('Browsers', 1), ('Utilities (General)', 2)])
 list([('Application/Video Players & Editors', 1), ('Application/Tools', 1)])
 list([('Media Players', 1), ('Customization & Ringtones', 1)])
 array(['792d078beb1e6f63223cf8316814378e598651c5',
        '7aa10a57509765894796e3fccff8d73d75ceac35'], dtype=object)
 list([('MX-SIN-Culiacán', 112), ('MX-JAL-Culiacán', 1), ('MX-JAL-Guadalajara', 34), ('MX--Tlalpan', 7), ('MX--Culiacán', 3), ('MX--Morelos', 1), ('MX-NLE-Tlalpan', 11), ('MX-SIN-Mazatlán', 2), ('MX-TAM-Valle Hermoso', 4), ('MX-CHH-Ciudad Juárez', 1)])
 list([('MX', 369)]) list([(

[0 0 0 0 0 0 0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 'fcce26ae1abe348cb24c4f4e542957ef5d9773fd' 'social networking'
 'entertainment' None None 'iq' None 'infinix' 'x6858' 'android' '15.0.0'
 '05' '2025_march' nan 5 3.5 None nan None
 list([('Application/Music & Audio', 1)]) list([('Media Players', 1)])
 None None
 array(['cd9b2558bcf52078c64242e751017419651181d9'], dtype=object)
 list([('IQ-NA-Najaf', 4), ('IQ-KA-Karbala', 3), ('IQ-BA-Basra', 39), ('IQ-BA-Baghdad', 13)])
 list([('IQ', 59)]) None None None None list([('ar', 59)])
 list([('15.0.0', 59)]) nan None None
 list([(0, 0.09259259259259259), (5, 0.037037037037037035), (10, 0.12962962962962962), (14, 0.1111111111111111), (20, 0.018518518518518517), (1, 0.018518518518518517), (6, 0.018518518518518517), (21, 0.07407407407407407), (13, 0.018518518518518517), (17, 0.018518518518518517), (12, 0.2037037037037037), (7, 0.018518518518518517), (18, 0.037037037037037035), (11, 0.12962962962962962), (8, 0.018518518518518517), (19, 0.0185

[0 0 0 0 0 0 0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 '7eaeef21e215654ea75a5ef78692d1f8ffd0ca54' 'sport betting' 'unknown' None
 None 'th' None 'samsung' 'sm-a156e' 'android' '14.0.0' '09'
 '2023_december' 299.0 5 nan None nan None None None None None None None
 None None None None None None None nan None None None None None None nan
 None None nan None None None None None None None None None None None None
 None None None None nan nan nan None None None None
 '060d9b16-d149-4428-aba4-3573992226b4' '2025-10-03-09-00']
[0 0 0 0 0 0 0 0.0 0.0 0.0 nan 0.0 0.0 0.0 0.0 0.0 0.0
 '001b729f996da29d01e9e3641ac7fb153e6bc98e' 'game' 'casual' 'match swap'
 None 'in' None 'oppo' 'cph2637' 'android' '15.0.0' '10' '2024_july' 357.0
 5 nan None nan None None None None None
 array(['1704071af5bb05db376ed785633f8d8d611f6ee7'], dtype=object)
 list([('IN-DL-New Delhi', 1), ('IN-JK-Jammu', 2)]) list([('IN', 3)]) None
 None None None list([('unknown', 3)]) list([('15.0.0', 3)]) nan None None
 list([(8, 0.

[0 0 0 0 0 0 0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0
 '3714e810963165f514bbeec2cc0b54e91239314b' 'application' 'dating'
 'dating' None 'dz' None 'infinix' 'x6525d' 'android' '14.0.0' '14'
 '2023_november' 74.0 5 nan None nan None None None None None None None
 None None None None None None None nan None None None None None None nan
 None None nan None None None None None None None None None None None None
 None None None None nan nan nan None None None None
 '6652d437-5371-43d6-887c-57d747a91a28' '2025-10-03-14-00']
